<a href="https://colab.research.google.com/github/miroorc/Capstone-Project/blob/main/Longformer_ecthr_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade datasets
!pip install -U accelerate
!pip install transformers
!pip install evaluate
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.1 MB/s eta 0:00:00


In [ ]:
#load the ecthra dataset from lex_glue
from datasets import load_dataset
ecthra = load_dataset("lex_glue", "ecthr_a")

Generating train split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
ecthra['train']=ecthra['train'].select(range(6000))
ecthra['test']=ecthra['test'].select(range(600))
ecthra['validation']=ecthra['validation'].select(range(600))

In [ ]:
ecthra

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 6000
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 600
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 600
    })
})

In [ ]:
#Extract the train, test, and validation parts of the ecthra dataset.
train=ecthra['train']
validation=ecthra['validation']
test=ecthra['test']

In [ ]:
#convert label to one-hot form
import numpy as np
num_classes = 10

def one_hot_encode_multi(labels, num_classes):
    encoded_labels = np.zeros((len(labels),num_classes))
    for i, sublist in enumerate(labels):
        encoded_labels[i, sublist] = 1
    return encoded_labels

In [ ]:
#call train, test, validation data and covnert it into one-hot-encoding vector with the function above
train_one_hot=one_hot_encode_multi(train['labels'],10)
train_one_hot = train_one_hot.tolist()
train=train.add_column(name="labels_one_hot", column=train_one_hot)

test_one_hot=one_hot_encode_multi(test['labels'],10)
test_one_hot = test_one_hot.tolist()
test=test.add_column(name="labels_one_hot", column=test_one_hot)

validation_one_hot=one_hot_encode_multi(validation['labels'],10)
validation_one_hot = validation_one_hot.tolist()
validation=validation.add_column(name="labels_one_hot", column=validation_one_hot)

In [ ]:
#clean the dataset to remove meaningless characters
import re
def format_text(text):
    text_format = []
    text = [[re.sub(r"^\d+\.\s*", "", s) for s in sublst] for sublst in text]
    for each in text:
        text_format.append(' '.join(each))
    prefix = 'Judge the following case: '
    text_format = [prefix + s for s in text_format]
    return text_format

In [ ]:
#clean the data (all train, test, validation) to have to correct format using the function above
train_text_format=format_text(train['text'])
train=train.remove_columns('text')
train=train.add_column(name="text", column=train_text_format)
ecthra['train']=train

test_text_format=format_text(test['text'])
test=test.remove_columns('text')
test=test.add_column(name="text", column=test_text_format)
ecthra['test']=test

validation_text_format=format_text(validation['text'])
validation=validation.remove_columns('text')
validation=validation.add_column(name="text", column=validation_text_format)
ecthra['validation']=validation

In [ ]:
ecthra=ecthra.remove_columns('labels')
ecthra=ecthra.rename_column('labels_one_hot', 'labels')

In [ ]:
# from transformers import AutoTokenizer, BartForSequenceClassification
from transformers import AutoTokenizer, RobertaModel
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
from transformers import LongformerTokenizer
from transformers import LongformerModel, AutoTokenizer

In [ ]:
#add fine-tuning model
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# model_ckpt = "facebook/bart-base"
# tokenizer = AutoTokenizer.from_pretrained(model_ckpt, problem_type="multi_label_classification")
# tokenizer = AutoTokenizer.from_pretrained("roberta-base")
# tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")
num_labels=10
model = AutoModelForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=num_labels)
# num_labels = 10
# model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=10)
# model = RobertaModel.from_pretrained("roberta-base", num_labels=num_labels, problem_type="multi_label_classification")


Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#function below is calling tokenizer to convert text into tokens
def tokenize_and_encode(examples):
  return tokenizer(examples["text"], truncation=True)#, padding='max_length', max_length=3000, return_tensors='pt')

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
# import torch

# tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=10)

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# print("input: ", inputs)
# outputs = model(**inputs)
# print("outputs: ", outputs)

# last_hidden_states = outputs.last_hidden_state

In [ ]:
cols = ecthra["train"].column_names
cols.remove("labels")
#call tokenize function into data
ds_enc = ecthra.map(tokenize_and_encode, batched=True, remove_columns=cols)


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:
ds_enc

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 6000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 600
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 600
    })
})

# before filter <4096 tokens to fit the limit


In [ ]:
#check the length of tokens of the first 10 elements before filtering it to be less than 4096 tokens
for i in range(0,10):
  print(len(ds_enc["test"]["input_ids"][i]))

4096
1305
415
806
2456
2019
2192
2943
1374
634


In [ ]:
#this function filters data so that it only takes in dataset that are less than 4096 tokens
def filter_rows(example):
    return len(example['input_ids']) < 4096 #filter so that it only includes dataset that fits everything within the limit

ds_enc = ds_enc.filter(filter_rows, load_from_cache_file=False)


Filter:   0%|          | 0/6000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/600 [00:00<?, ? examples/s]

Filter:   0%|          | 0/600 [00:00<?, ? examples/s]

# after filter < 4096 tokens

In [ ]:
#check the length of tokens of the first 10 elements after filtering it to be less than 4096 tokens
for i in range(0,10):
  print(len(ds_enc["test"]["input_ids"][i]))

1305
415
806
2456
2019
2192
2943
1374
634
234


In [ ]:
print(len(ds_enc["train"]["input_ids"]))
print(len(ds_enc["test"]["input_ids"]))
print(len(ds_enc["validation"]["input_ids"]))

5465
512
517


In [ ]:
ds_enc

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5465
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 512
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 517
    })
})

In [ ]:
ds_enc.set_format("torch")

In [ ]:
# num_labels=10
# model = BartForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels, problem_type="multi_label_classification")

In [ ]:
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)

In [ ]:
# model.to('cuda')

In [ ]:
# define the arguments
batch_size = 1
metric_name = "f1"

args = TrainingArguments(
    f"output",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=1,
    num_train_epochs=4,
    weight_decay=0.1,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score,recall_score
from transformers import EvalPrediction
import torch

from sklearn.metrics import accuracy_score, f1_score
import numpy as np

#function below returns accuracy, f1 score, and recall of the model.
def multi_label_metrics(predictions, labels):
    # Convert logits to 0 or 1
    print("predictions before threshold: ", predictions)
    #filter so that it turns prediction to 1 only when prediction exceeds 80%
    predictions = np.where(predictions > 0.8, 1, 0)

    # Compute metrics
    accuracy = accuracy_score(y_true=labels, y_pred=predictions)
    f1 = f1_score(y_true=labels, y_pred=predictions, average='samples')
    recall = recall_score(y_true=labels, y_pred=predictions, average='macro')
    return {
        'accuracy': accuracy,
        'f1': f1,
        'recall': recall
    }

import torch

#helper function that calls multi_label_metrics function and produces prediction by applying softmax
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    #apply softmax
    predictions = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
    return multi_label_metrics(predictions, labels)

from torch.nn import BCEWithLogitsLoss

from transformers import Trainer
from torch.nn import BCEWithLogitsLoss


# class and function below inputs the token to the model, produces output and calculates its loss
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = BCEWithLogitsLoss()
        loss = loss_fct(logits.view(-1, model.config.num_labels), labels.float().view(-1, model.config.num_labels))

        return (loss, outputs) if return_outputs else loss

#define trainer with arguments, models, and datasets
trainer = CustomTrainer(
    model=model,
    args=args,
    train_dataset=ds_enc["train"],
    eval_dataset=ds_enc["validation"],
    compute_metrics=compute_metrics,
)









In [ ]:
import torch
torch.cuda.empty_cache()
trainer.evaluate()

predictions before threshold:  [[0.07914229 0.10642043 0.09645908 ... 0.09219283 0.09361117 0.10651696]
 [0.08748211 0.10090584 0.09408876 ... 0.0877462  0.09926549 0.10336682]
 [0.08624303 0.09952783 0.09585601 ... 0.08495122 0.09938344 0.09962257]
 ...
 [0.07796297 0.10674048 0.09568875 ... 0.09250364 0.09320219 0.10711519]
 [0.08317953 0.10255207 0.10039551 ... 0.08989798 0.09247506 0.10745927]
 [0.07773336 0.1062264  0.09613427 ... 0.09294219 0.0930354  0.10743602]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


{'eval_loss': 0.7182280421257019,
 'eval_accuracy': 0.17408123791102514,
 'eval_f1': 0.0,
 'eval_recall': 0.0,
 'eval_runtime': 1595.5625,
 'eval_samples_per_second': 0.324,
 'eval_steps_per_second': 0.324}

In [ ]:
trainer.train()
#epoch 3 LR 1e-5 prediction > 0.5 --> 0.545
#epoch 3 LR 1e-5 prediction > 0.7 --> 0.544 make epoch larger
#epoch 5 LR 1e-5 prediction > 0.7 --> 0.546
#epoch 6 LR 1e-4 prediction > 0.8 --> 0.175
#epoch 6 LR 1e-6 prediction > 0.8 --> 0.529
#epoch 6 LR 1e-5 prediction > 0.8 -->


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall
1,0.148700,0.260757,0.402321,0.448098,0.262575


predictions before threshold:  [[1.81148537e-02 1.36970337e-02 1.04228891e-02 ... 4.44678441e-02
  5.84064908e-02 5.62956966e-02]
 [9.05329129e-04 9.62455034e-01 3.36573236e-02 ... 2.04499229e-04
  1.99469228e-04 3.34745360e-04]
 [2.02665641e-03 8.41703713e-02 1.40144140e-03 ... 3.35497141e-04
  8.67826573e-04 2.42970511e-03]
 ...
 [4.09354980e-05 1.73805121e-04 1.01680183e-04 ... 1.97280388e-05
  5.42417401e-05 4.21006145e-04]
 [6.45380278e-05 1.99654241e-04 1.25643375e-04 ... 2.84929374e-05
  1.03819584e-04 8.60905100e-04]
 [1.78776008e-05 5.98174229e-05 4.20854922e-05 ... 1.37480529e-05
  2.56919702e-05 4.53271605e-02]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [ ]:
#evaluate after training
trainer.evaluate()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

confusion_matrices = x['eval_matrixes']

labels = ['Positive', 'Negative']

fig, axs = plt.subplots(2, 5, figsize=(15, 6))
for i, ax in enumerate(axs.flatten()):
    sns.heatmap(confusion_matrices[i], annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels, ax=ax)
    ax.set_title(f'Confusion Matrix {i+1}')
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('Actual Label')

plt.tight_layout()
plt.show()